In [1]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import re
import os

In [2]:
## Loading data - an excell file :

data1 = pd.ExcelFile('Tranzact2.xlsx')
tranzact_data1 = data1.parse(0)

lamba = len(tranzact_data1)
tranzact_data1.columns

Index(['Company', 'Supplier', 'Item', 'Category', 'UOM', 'Unnamed: 5',
       'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14',
       'Unnamed: 15', 'Unnamed: 16'],
      dtype='object')

In [3]:
tranzact_data2 = tranzact_data1.iloc[0:lamba, 0:3]    ## Remove unnecessary columns
tranzact_data2['Item'] = tranzact_data2['Item'].map(lambda x: x if type(x)!=str else x.lower()) ## make Items in lower case
tranzact_data = tranzact_data2.dropna()              ## remove rows with cell value none
tranzact_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28921 entries, 0 to 28921
Data columns (total 3 columns):
Company     28921 non-null object
Supplier    28921 non-null object
Item        28921 non-null object
dtypes: object(3)
memory usage: 903.8+ KB


In [4]:
tranzact_data.head()

,Company,Supplier,Item
0,M/S Plastech,Pukhraj Additives LLP,zincoplast zinc
1,M/S Plastech,Pukhraj Additives LLP,zincoplast pyn
2,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup..."
3,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup..."
4,Modern Engineering & Spring Company,Bijal Steels,"chromium manganese spring steel as rolled, sup..."


In [5]:
## Cleaning the Item Description to create a feature matrix :

pd.options.mode.chained_assignment = None

tranzact_data['features'] = tranzact_data['Item'].str.replace(r'\b\d+\b','')  ## remove integers
print(tranzact_data['features'][5])

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\W',' ')   ## remove puntuations
print(tranzact_data['features'][5])

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\b\d+\mm\b',' ')   ## remove all 'mm' dimensions only
print(tranzact_data['features'][5])

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\b\w\b','') ## remove stand alone single letters
print(tranzact_data['features'][5])

tranzact_data['features'] = tranzact_data['features'].str.replace(r'\s+',' ') ## remove gaps between words to singe gap
print(tranzact_data['features'][5])


## Removing unneccessary words like measurment units etc.
                                                                        
words = stopwords.words("english")

words.append('mm')
words.append('ft')
words.append('as')
words.append('to')
words.append('nos')

tranzact_data['features']= tranzact_data['features'].apply(lambda x:' '.join([i for i in x.split()
                                                                                if i not in words]).lower())
print(tranzact_data['features'][5])

ms sheet  ft x  ft x . mm -  nos
ms sheet  ft x  ft x   mm    nos
ms sheet  ft x  ft x   mm    nos
ms sheet  ft   ft    mm    nos
ms sheet ft ft mm nos
ms sheet


In [6]:
## Taking out the new file:

writer = pd.ExcelWriter('Tranzact2_features.xlsx', engine='xlsxwriter')

tranzact_data.to_excel(writer, sheet_name='Sheet1')

writer.save()